In [1]:
import csv
import pandas as pd
from pyinflect import getAllInflections, getInflection
from copy import deepcopy

In [3]:
#pd.show_versions()
#print(pd.__version__)

In [4]:
#set the mode to view the df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [5]:
### THIS IS AN ORIGINAL NGRAM WITH POSSIBLE DUPLICATES. If you read this file, clean the data with extra steps: 
ngram = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Transitivity_from_GoogleNgram_v1.xlsx', index_col=None)
ngram = ngram.set_index('entry')
entrylist = ngram.index.tolist() #list of all entries in Google Ngram spreadsheet

### find the duplicate member of entry (Warning: It takes A WHILE)
duplicates = [i for i in entrylist if entrylist.count(i)!=1]
ngram_clean = ngram[~ngram.index.duplicated()]
del ngram

### save the new ngram to a file (Next time: just start from this file)
# ngram2.to_csv('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Transitivity_from_GoogleNgram_NoDuplicate.csv')

In [22]:
### IF Ngram spreadsheet already exists and hasn't been changed, READ THIS FILE INSTEAD
ngram_clean = pd.read_csv('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Transitivity_from_GoogleNgram_NoDuplicate.csv', index_col=None)

In [35]:
ngram_dict = ngram_clean.to_dict(orient='index') ##ngram as dictionary

In [37]:
##load verbvector spreadsheet
vv = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/verblist_v2.13.xlsx', index_col='Lemma')

In [72]:
vv_dict = vv.to_dict(orient='index')
verblist = list(vv_dict.keys()) ##get the list of all verbs

In [73]:
vv_dict['shuffle'].values()

dict_values([42, 42, 7.58, 3.83, 8.95, 'VerbNet', 'both', nan, 'unergative?', 'unergative', nan, nan, 1.0, 1.0, 'tr', nan, nan, nan, nan, nan, 'tr', 'unergative', 0.5464, 0.4471, 'Process', 'body mvt or posture', 'body movement or posture', 4.0, 'body movement or posture/obj dir action', 4.0, 'manner of motion', 'manner of motion', 'verbs of combining attaching, verbs of involving the body, verbs of motion, verbs of motion', '22.3shake, 40.3.2crane, 51.3.2run, 51.5waltz', 0.71, 7, 3, 5, 2.3, 1.7, 4, 2, 632.206, 1.0, 646.607, 1.0, 270.96, 15.17])

In [74]:
## convert the unerga_pb, unacc_pb, tr_pb, intr_g, tr_g as percentage
for key, value in vv_dict.items():
    for k, v in value.items():
        if k in ['unerg_pb', 'unacc_pb', 'tr_pb', 'intr_g', 'tr_g', 'intr_google', 'trans_google'] and v != 'notfound':
            #v_new = float(v*100)
            v_new = round(float(v*100), 2)
            vv_dict[key][k] = v_new

In [76]:
vv_dict['shuffle'].values()

dict_values([42, 42, 7.58, 3.83, 8.95, 'VerbNet', 'both', nan, 'unergative?', 'unergative', nan, nan, 100.0, 1.0, 'tr', nan, nan, nan, nan, nan, 'tr', 'unergative', 54.64, 44.71, 'Process', 'body mvt or posture', 'body movement or posture', 4.0, 'body movement or posture/obj dir action', 4.0, 'manner of motion', 'manner of motion', 'verbs of combining attaching, verbs of involving the body, verbs of motion, verbs of motion', '22.3shake, 40.3.2crane, 51.3.2run, 51.5waltz', 0.71, 7, 3, 5, 2.3, 1.7, 4, 2, 632.206, 1.0, 646.607, 1.0, 270.96, 15.17])

In [75]:
#verblist
vv_dict['work']

{'GrandIndex': 1,
 'Index': 1,
 'LogFreqHAL': 13.02,
 'Concreteness': 3.48,
 'AgeofAcqsn': 5.86,
 'Database': 'CREA',
 'Trans_Intrans': 'both',
 'SynClassLitrt': 'unergative',
 'SynClassMemo': 'unergative',
 'SynClassSK': 'unergative',
 'unerg_pb': 78.0,
 'unacc_pb': 4.4,
 'tr_pb': 16.0,
 'totalN_pb': 431.0,
 'most_frequent_pb': 'unerg',
 'intr_g': nan,
 'tr_g': nan,
 'totalN_g': nan,
 'most_frequent_g': nan,
 'Note': nan,
 'SynClassCorpus': 'unerg',
 'SynClassFinal': 'unergative',
 'intr_google': 88.78,
 'trans_google': 11.17,
 'AspClass_Consensus': 'Process',
 'SemanticType_JB': 'other - general process',
 'SemanticType_JB_edit': 'other - general process',
 'Typicality_JB': 5.0,
 'SemanticType_LC': 'other',
 'Typicality_LC': nan,
 'SemClass_Consensus': 'other',
 'CONSENSUS_JB': 'other',
 'SemanticTypeLevin': 'verbs of creating and transformation',
 'SemanticTypeEntry': '26.5knead',
 'Percentage_dom_PoS': 0.57,
 'Length': 4,
 'Ortho_N': 9,
 'Phono_N': 25,
 'OLD': 1.5,
 'PLD': 1.2,
 'N

In [88]:
### construct an inflection dictionary, i.e., {run: ['runs, ran, running']}
inflection_dict = {}
for verb in verblist:
    inflection_dict[verb] = []
    inflections = getAllInflections(verb, pos_type='V')
    for k, v in inflections.items():
        v_list = list(v) ##tuple to list
        if not v_list[0].endswith('ing'):
            inflection_dict[verb].append(v_list[0]) ### if there are >1 variant for a given inflection, only keep 1st
    liszt = inflection_dict[verb] ##to drop duplicates, convert it into a set
    inflection_dict[verb] = list(set(liszt))

inflection_dict['arrive']

['arrives', 'arrived', 'arrive']

In [ ]:
### for testing
# trans_count = 0
# intrans_count = 0
# total_count = 0 
# for wordform in inflection_dict['work']:
#     print (wordform, ngram_dict[wordform])
#     trans_count += ngram_dict[wordform]['transitive']
#     intrans_count += ngram_dict[wordform]['intransitive']
#     total_count += ngram_dict[wordform]['totalcount']
#     trans_percent = round((trans_count/total_count)*100, 2)
#     intrans_percent = round((intrans_count/total_count)*100,2)

# print(trans_count)
# print(intrans_count)
# print(total_count)
# print(trans_percent)
# print(intrans_percent)

In [89]:
## update the vv dictionary (our verb vector)
for k, v in vv_dict.items(): ### i.e., k='work', v={'GrandIndex':1, ...}
    trans_count = 0
    intrans_count = 0
    total_count = 0
    wordforms = inflection_dict[k] ##wordforms = ['work', 'works', 'working', ...]
    for wordform in wordforms: 
        try: 
            trans_count += ngram_dict[wordform]['transitive']
            intrans_count += ngram_dict[wordform]['intransitive']
            total_count += ngram_dict[wordform]['totalcount']
            trans_percent = round((trans_count/total_count)*100, 2)
            intrans_percent = round((intrans_count/total_count)*100, 2)    
        except: 
            print (wordform, wordform in ngram_dict.keys())
    v['google1950_trans'] = trans_percent
    v['google1950_intrans'] = intrans_percent

oinks False


In [90]:
vv_dict['weaken']

{'GrandIndex': 187,
 'Index': 98,
 'LogFreqHAL': 7.114,
 'Concreteness': 2.37,
 'AgeofAcqsn': 7.95,
 'Database': 'VerbNet',
 'Trans_Intrans': 'both',
 'SynClassLitrt': nan,
 'SynClassMemo': 'unaccusative',
 'SynClassSK': 'unaccusative',
 'unerg_pb': nan,
 'unacc_pb': 65.0,
 'tr_pb': 33.0,
 'totalN_pb': 55.0,
 'most_frequent_pb': 'unacc',
 'intr_g': nan,
 'tr_g': nan,
 'totalN_g': nan,
 'most_frequent_g': nan,
 'Note': nan,
 'SynClassCorpus': 'unacc',
 'SynClassFinal': 'unaccusativefailed',
 'intr_google': 24.98,
 'trans_google': 75.01,
 'AspClass_Consensus': 'Process',
 'SemanticType_JB': 'ch of state',
 'SemanticType_JB_edit': 'ch of state',
 'Typicality_JB': 5.0,
 'SemanticType_LC': 'ch of state',
 'Typicality_LC': 4.0,
 'SemClass_Consensus': 'ch of state',
 'CONSENSUS_JB': 'ch of state',
 'SemanticTypeLevin': 'verbs of change of state',
 'SemanticTypeEntry': '45.4state',
 'Percentage_dom_PoS': 1.0,
 'Length': 6,
 'Ortho_N': 1,
 'Phono_N': 6,
 'OLD': 1.85,
 'PLD': 1.7,
 'NPhon': 5,
 

In [91]:
##save vv_dict to a df
df_out = pd.DataFrame.from_dict(vv_dict, orient='index')

In [92]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

In [93]:
df_out2 = rearrange_col(df_out, 'google1950_intrans', 'trans_google')
df_out3 = rearrange_col(df_out2, 'google1950_trans', 'google1950_intrans')
df_out3.head()

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,google1950_intrans,google1950_trans,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
work,1,1,13.020,3.48,5.86,CREA,both,unergative,unergative,unergative,78.0,4.40,16.0,431.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,88.78,11.17,90.12,9.88,Process,other - general process,other - general process,5.0,other,NaN,other,other,verbs of creating and transformation,26.5knead,0.57,4,9,25,1.50,1.20,3,1,633.152,1.00,521.889,1.000,2421.10,1034.67
go,2,2,12.923,3.15,3.37,CREA,intransitive only,unergative,unergative PP arg,unergative,16.0,43.00,2.0,1029.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,unergative,95.67,4.32,96.44,3.56,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of existence, verbs of motion","47.7meander, 51.1advance",0.98,2,17,48,1.35,1.00,2,1,597.688,0.97,524.071,1.000,974.49,NaN
run,3,3,12.316,4.31,4.47,CREA,both,unergative,unergative,unergative,18.0,34.00,44.0,374.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unergative,76.27,23.65,72.44,27.56,Process,body mvt or posture,body movement or posture,5.0,manner of motion,5.0,manner of motion,manner of motion,"verbs of creating and transformation, verbs of existence, verbs of existence, verbs of motion","26.3prepare, 47.5.1swarm, 47.7meander, 51.3.2run",0.89,3,19,40,1.00,1.00,3,1,596.647,1.00,559.955,1.000,386.53,229.70
play,4,4,11.915,3.24,4.10,CREA,both,unergative,"unergative, transitive",unergative,9.7,15.00,72.0,186.0,tr,33.0,57.0,150.0,tr,NaN,tr,unergative,55.61,44,48.91,51.09,Process,social event,social event,5.0,social or interpersonal event,3.0,social or interpersonal event,social or interpersonal event,"verbs of creating and transformation, verbs of social interaction","26.7perform, 36.3meet",0.81,4,6,15,1.45,1.10,3,1,537.909,1.00,566.077,0.963,781.22,329.47
live,5,5,11.763,3.57,6.10,CREA,intransitive only,unergative/unaccusative,unergative PP arg,unergative,95.0,0.78,4.7,129.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,94.18,5.81,92.50,7.50,State,existence,existence,5.0,existence,5.0,existence,existence,"verbs of ingesting, lodge verbs, verbs of existence","39.6gorge, 46lodge, 47.1exist",0.89,4,16,21,1.15,1.25,3,1,629.886,1.00,553.769,0.963,3645.64,606.44


In [94]:
## save the df to a csv
df_out3.to_csv('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/verblist_v2.14.csv')